In [ ]:
require "bundler/inline"
gemfile do
  source "https://rubygems.org"
  gem "awesome_print"
  gem "json"
end

In [ ]:
data = JSON.parse(File.read("result.json"))

In [ ]:
ap data["messages"][0..10]
messages = data["messages"]; :ok

In [8]:
filtered_messages = messages
  .select { _1["type"] == "message" } # only messages
  .select { _1["from_id"] } # only messages from actual people
  .select { _1["text"] && _1["text"] != "" };1 # only messages with text

1

In [30]:
ap filtered_messages[-50..-1]

[
    [ 0] {
                         "id" => 46128,
                       "type" => "message",
                       "date" => "2024-01-23T20:11:22",
              "date_unixtime" => "1706051482",
                     "edited" => "2024-01-23T20:52:05",
            "edited_unixtime" => "1706053925",
                       "from" => "Marlos",
                    "from_id" => "user21385579",
        "reply_to_message_id" => 44150,
                       "text" => "Aqui tamb\u00E9m. Api, graphql e react. Nunca testei o hotwire. Na real acho ate confuso hotwire, importmap, stimulus, nao saquei bem qual\u00E9..no unico monolito que mexo ainda \u00E9 com sprockets",
              "text_entities" => [
            [0] {
                "type" => "plain",
                "text" => "Aqui tamb\u00E9m. Api, graphql e react. Nunca testei o hotwire. Na real acho ate confuso hotwire, importmap, stimulus, nao saquei bem qual\u00E9..no unico monolito que mexo ainda \u00E9 com sprockets"
            }

In [ ]:
ap filtered_messages.find { _1["id"] == 45708 }

In [ ]:
# Let's see how to treat these pesky text arrays
types = Set.new
filtered_messages.each do
  next unless _1["text"].is_a?(Array)

  _1["text"].each do |element|
    next unless element.is_a?(Hash)

    if !types.include?(element["type"])
      types << element["type"]
      ap element
    end
  end; 1
end; 1

I'll just extract the plain text from the entities and join all the text together.

In [56]:
def handle_text(text_object)
  return text_object if text_object.is_a?(String)

  text_object.map do |element|
    next element if element.is_a?(String)

    element["text"]
  end.join
end

mapped_messages = filtered_messages.map do
  {
    id: _1["id"],
    sent_at: _1["date"],
    author: _1["from"],
    author_id: _1["from_id"],
    reply_to: _1["reply_to_message_id"],
    text: handle_text(_1["text"])
  }
end; 1

1

In [57]:
ap mapped_messages[-50..-1]

[
    [ 0] {
               :id => 46128,
          :sent_at => "2024-01-23T20:11:22",
           :author => "Marlos",
        :author_id => "user21385579",
         :reply_to => 44150,
             :text => "Aqui tamb\u00E9m. Api, graphql e react. Nunca testei o hotwire. Na real acho ate confuso hotwire, importmap, stimulus, nao saquei bem qual\u00E9..no unico monolito que mexo ainda \u00E9 com sprockets"
    },
    [ 1] {
               :id => 46129,
          :sent_at => "2024-01-23T20:51:31",
           :author => "Rodrigo Serradura",
        :author_id => "user258199043",
         :reply_to => 44150,
             :text => "pessoal, passando aqui pra compartilhar um compiladinho de materiais que eu passei no canal off-topic aqui que t\u00E1 aprendendo Ruby:\n\nCurso de Ruby do Onebit Code: https://www.youtube.com/playlist?list=PLdDT8if5attEOcQGPHLNIfnSFiJHhGDOZ\n\nApostila da Alura sobre HTML, CSS e Javascript: https://www.alura.com.br/apostila-html-css-javascript\n\nApostila da Al

In [58]:
File.write("clean_messages.json", JSON.generate(mapped_messages))

9180208